<a href="https://colab.research.google.com/github/Pavel184/NLP_basic_course/blob/Homework_lesson_7/Homework_lesson_7_last_rev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### HOMEWORK_LESSON_7

1. Учим conv сеть для классификации

In [1]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 8.2 MB 1.3 MB/s 


In [2]:
!pip install stop-words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=9ca2109ffe76397f4f8ebaa106e3f1e6f84425de85f4bd76c62d92b591f93749
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [3]:
import pandas as pd
import numpy as np
import keras
from keras import utils as np_utils
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, Concatenate
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
#from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  
from sklearn import model_selection, preprocessing, linear_model
import tensorflow as tf
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.probability import FreqDist
import os
from datetime import datetime, date

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
from google.colab import drive

drive.mount('/content/drive')  

Mounted at /content/drive


In [5]:
# Загрузка датасета
df_data = pd.read_excel('/content/drive/My Drive/Colab Notebooks/NLP_basic_course/datasets/отзывы за лето.xls')

In [6]:
# Перевод в тип строка всех отзывов
df_data['Content'] = df_data['Content'].astype(str)

In [7]:
# Перевод даты в количество дней от '2017-01-01'
df_data['Date'] = pd.to_datetime(df_data['Date'])
df_data['Days'] = (df_data['Date'] - datetime.strptime('2017-01-01', '%Y-%m-%d')).dt.days

In [8]:
# Просмотр датасета
df_data

,Rating,Content,Date,Days
0,5,It just works!,2017-08-14,225
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,225
2,5,Отлично все,2017-08-14,225
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,225
4,5,"Очень удобно, работает быстро.",2017-08-14,225
...,...,...,...,...
20654,1,"Ну и шляпа,с роот правами бесполезная прога,ра...",2017-06-01,151
20655,5,Ок,2017-06-01,151
20656,4,Доволен,2017-06-01,151
20657,1,"Песопаснасть, рут ни нужын",2017-06-01,151


In [9]:
# Для дальнейшей работы берём отзывы и количество дней от '2017-01-01'
X_col_list = ['Content', 'Days'] 

In [10]:
# Разделение на тренировочную и тестовую выборки
train_x, test_x, train_y, test_y = model_selection.train_test_split(df_data[X_col_list], df_data['Rating'])

In [11]:
train_x

,Content,Days
4756,"Удобно, просто!",218
6666,Зачем загадили приложение? Более года пользуюс...,215
1805,Проверка штрафов-каждый раз надо заново вводит...,222
1595,После обновления стали неверно отображаться да...,222
724,Все хорошо.,224
...,...,...
12481,Не устанавливается пишет код ошибки - 24,207
5470,Удобное приложение,217
19725,В приложение возможно зайти только с 3-го раза...,174
270,Отличное приложение,224


In [12]:
# Очистка текста
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

train_x['Content'] = train_x['Content'].apply(preprocess_text)
test_x['Content'] = test_x['Content'].apply(preprocess_text)

In [13]:
# Перевод в нижний регистр
train_corpus = " ".join(train_x["Content"])
train_corpus = train_corpus.lower()

In [14]:
# Токенизация
tokens = word_tokenize(train_corpus)

In [15]:
# Фильтрация
tokens_filtered = [word for word in tokens if word.isalnum()]

In [16]:
max_words = 10584
max_len = 40
num_classes = 5

# Training
epochs = 10
batch_size = 512
print_batch_n = 100

In [17]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [18]:
# Словарь
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [19]:
# Функция перевода  последовательность
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [20]:
#  Тренировочная и тестовая последовательности
x_train = np.asarray([text_to_sequence(text, max_len) for text in train_x['Content']], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in test_x['Content']], dtype=np.int32)

In [21]:
# OHE
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [22]:
# Тензоры с метками классов
num_classes = 5
y_train = keras.utils.np_utils.to_categorical(train_y, num_classes)
y_test = keras.utils.np_utils.to_categorical(test_y, num_classes)

In [23]:
# Модель с учётом дат отзывов
input_1 = tf.keras.Input(shape=(max_len))
input_2 = tf.keras.Input(shape=(1,))
emb_1 = Embedding(input_dim=max_words, output_dim=128, input_length=max_len)(input_1)
conv_1 = Conv1D(128, 3, activation='relu')(emb_1)
glob_max_pool = GlobalMaxPool1D()(conv_1)
dens_1 = Dense(10, activation='relu')(glob_max_pool)
dens_2 = Dense(10, activation='relu')(input_2)
combined = Concatenate()([dens_1, dens_2])

output = Dense(num_classes,activation='softmax' )(combined)

model = Model(inputs=[input_1,input_2], outputs=output)
model.compile(optimizer='SGD', loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [24]:
history = model.fit([x_train, train_x['Days']], y_train,
                    batch_size=batch_size,
                    epochs=15,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/15
28/28 [==============================] - 6s 196ms/step - loss: 16.5289 - accuracy: 0.1142 - val_loss: 5.6733 - val_accuracy: 0.7161
Epoch 2/15
28/28 [==============================] - 5s 170ms/step - loss: 6.0953 - accuracy: 0.7100 - val_loss: 4.3316 - val_accuracy: 0.7168
Epoch 3/15
28/28 [==============================] - 5s 165ms/step - loss: 4.2332 - accuracy: 0.7109 - val_loss: 3.2271 - val_accuracy: 0.7168
Epoch 4/15
28/28 [==============================] - 5s 161ms/step - loss: 3.0164 - accuracy: 0.7169 - val_loss: 2.1047 - val_accuracy: 0.7432
Epoch 5/15
28/28 [==============================] - 5s 179ms/step - loss: 1.5445 - accuracy: 0.7463 - val_loss: 0.9938 - val_accuracy: 0.6477
Epoch 6/15
28/28 [==============================] - 5s 177ms/step - loss: 0.8250 - accuracy: 0.7400 - val_loss: 0.8517 - val_accuracy: 0.7245
Epoch 7/15
28/28 [==============================] - 5s 166ms/step - loss: 0.7197 - accuracy: 0.7702 - val_loss: 0.8219 - val_accuracy: 0.7445
Epoch

In [25]:
score = model.evaluate([x_test, test_x['Days']], y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

11/11 [==============================] - 1s 44ms/step - loss: 0.9236 - accuracy: 0.7429


Test loss: 0.9235877990722656
Test accuracy: 0.7428848147392273


In [26]:
# Модель без учёта дат отзывов
model_1 = Sequential()
model_1.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model_1.add(Conv1D(128, 3))
model_1.add(Activation("relu"))
model_1.add(GlobalMaxPool1D())
model_1.add(Dense(10))
model_1.add(Activation("relu"))
model_1.add(Dense(num_classes))
model_1.add(Activation('softmax'))

In [27]:
# Сборка модели
model_1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [28]:
'''tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  '''

# Тренировка модели
history_1 = model_1.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/10
28/28 [==============================] - 6s 187ms/step - loss: 1.2395 - accuracy: 0.6865 - val_loss: 0.9825 - val_accuracy: 0.7123
Epoch 2/10
28/28 [==============================] - 5s 184ms/step - loss: 0.8903 - accuracy: 0.7082 - val_loss: 0.7768 - val_accuracy: 0.7277
Epoch 3/10
28/28 [==============================] - 5s 183ms/step - loss: 0.6928 - accuracy: 0.7610 - val_loss: 0.6763 - val_accuracy: 0.7606
Epoch 4/10
28/28 [==============================] - 5s 176ms/step - loss: 0.6037 - accuracy: 0.7883 - val_loss: 0.6552 - val_accuracy: 0.7729
Epoch 5/10
28/28 [==============================] - 4s 159ms/step - loss: 0.5505 - accuracy: 0.8065 - val_loss: 0.6525 - val_accuracy: 0.7787
Epoch 6/10
28/28 [==============================] - 5s 169ms/step - loss: 0.5013 - accuracy: 0.8267 - val_loss: 0.6620 - val_accuracy: 0.7819
Epoch 7/10
28/28 [==============================] - 5s 171ms/step - loss: 0.4528 - accuracy: 0.8503 - val_loss: 0.6823 - val_accuracy: 0.7748
Epoch 

In [29]:
score_1 = model_1.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_1[0])
print('Test accuracy:', score_1[1])

11/11 [==============================] - 0s 36ms/step - loss: 0.7975 - accuracy: 0.7541


Test loss: 0.7975063323974609
Test accuracy: 0.7541142106056213


2. Рассмотреть 2-а варианта сеточек

2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

In [30]:
# Список текстов
texts = []  # list of text samples
df_data_prep = df_data['Content'].apply(preprocess_text)
for text in df_data_prep:
  texts.append(text)

In [31]:
#labels_index = {}  # dictionary mapping label name to numeric id


In [32]:
# Количество отзывов
print('Found %s texts.' % len(texts))

Found 20659 texts.


In [33]:
VALIDATION_SPLIT = 0.3
# Токенизация
tokenizer = Tokenizer(nb_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
# Подготовка тензоров
data = pad_sequences(sequences, maxlen=max_len)
df_data_y = encoder.fit_transform(df_data['Rating'])
labels = keras.utils.np_utils.to_categorical(df_data_y, num_classes)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train_weight = data[:-nb_validation_samples]
y_train_weight = labels[:-nb_validation_samples]
x_val_weight = data[-nb_validation_samples:]
y_val_weight = labels[-nb_validation_samples:]

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 13485 unique tokens.
Shape of data tensor: (20659, 40)
Shape of label tensor: (20659, 5)


In [34]:
# Загрузка предобученных эмбеддингов tayga_upos_skipgram_300_2_2019
embeddings_index = {}
f = open(os.path.join('/content/drive/My Drive/Colab Notebooks/NLP_basic_course/datasets/pretrain/', 'model.txt'))
for line in f:
    values = line.split()
    word_and_ner = values[0].split('_')
    word = word_and_ner[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 195436 word vectors.


In [35]:
# Подготовка матрицы весов
EMBEDDING_DIM = 300
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [36]:
# Слой эмендинг
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

In [37]:
# Слои модели
model_2 = Sequential()
model_2.add(Input(shape=(max_len,), dtype='int32'))
model_2.add(embedding_layer)
model_2.add(Conv1D(128, 3))
model_2.add(Activation("relu"))
model_2.add(GlobalMaxPool1D())
model_2.add(Dense(10))
model_2.add(Activation("relu"))
model_2.add(Dense(num_classes))
model_2.add(Activation('softmax'))

In [38]:
# Компиляция модели
model_2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [39]:
# Обучение модели
model_2.fit(x_train_weight, y_train_weight, validation_data=(x_val_weight, y_val_weight),
          epochs=10, batch_size=128)

Epoch 1/10
113/113 [==============================] - 10s 80ms/step - loss: 0.8515 - accuracy: 0.7392 - val_loss: 0.7154 - val_accuracy: 0.7626
Epoch 2/10
113/113 [==============================] - 9s 77ms/step - loss: 0.6601 - accuracy: 0.7783 - val_loss: 0.6790 - val_accuracy: 0.7710
Epoch 3/10
113/113 [==============================] - 9s 77ms/step - loss: 0.5853 - accuracy: 0.7984 - val_loss: 0.6703 - val_accuracy: 0.7680
Epoch 4/10
113/113 [==============================] - 8s 74ms/step - loss: 0.5100 - accuracy: 0.8332 - val_loss: 0.6651 - val_accuracy: 0.7694
Epoch 5/10
113/113 [==============================] - 9s 77ms/step - loss: 0.4408 - accuracy: 0.8647 - val_loss: 0.6856 - val_accuracy: 0.7662
Epoch 6/10
113/113 [==============================] - 8s 72ms/step - loss: 0.3795 - accuracy: 0.8902 - val_loss: 0.7285 - val_accuracy: 0.7555
Epoch 7/10
113/113 [==============================] - 8s 73ms/step - loss: 0.3270 - accuracy: 0.9104 - val_loss: 0.7373 - val_accuracy: 0.761

In [40]:
score_2 = model_2.evaluate(x_val_weight, y_val_weight, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_2[0])
print('Test accuracy:', score_2[1])

13/13 [==============================] - 1s 92ms/step - loss: 0.8352 - accuracy: 0.7544


Test loss: 0.8351878523826599
Test accuracy: 0.7543972730636597


2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [41]:
model_3 = Sequential()
model_3.add(Embedding(input_dim=max_words, output_dim=128))
model_3.add(Conv1D(128, 3))
model_3.add(Activation("relu"))
model_3.add(GlobalMaxPool1D())
model_3.add(Dense(10))
model_3.add(Activation("relu"))
model_3.add(Dense(num_classes))
model_3.add(Activation('softmax'))

In [42]:
model_3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [43]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model_3.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/10
28/28 [==============================] - 6s 184ms/step - loss: 1.2788 - accuracy: 0.6865 - val_loss: 1.0674 - val_accuracy: 0.7123
Epoch 2/10
28/28 [==============================] - 5s 163ms/step - loss: 0.9924 - accuracy: 0.7073 - val_loss: 0.8616 - val_accuracy: 0.7161
Epoch 3/10
28/28 [==============================] - 5s 179ms/step - loss: 0.8038 - accuracy: 0.7202 - val_loss: 0.7565 - val_accuracy: 0.7432
Epoch 4/10
28/28 [==============================] - 5s 176ms/step - loss: 0.6908 - accuracy: 0.7742 - val_loss: 0.6931 - val_accuracy: 0.7658
Epoch 5/10
28/28 [==============================] - 5s 175ms/step - loss: 0.5987 - accuracy: 0.8047 - val_loss: 0.6658 - val_accuracy: 0.7690
Epoch 6/10
28/28 [==============================] - 4s 159ms/step - loss: 0.5261 - accuracy: 0.8274 - val_loss: 0.6621 - val_accuracy: 0.7723
Epoch 7/10
28/28 [==============================] - 4s 159ms/step - loss: 0.4609 - accuracy: 0.8470 - val_loss: 0.6811 - val_accuracy: 0.7652
Epoch 

In [44]:
score_3 = model_3.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score_3[0])
print('Test accuracy:', score_3[1])

11/11 [==============================] - 0s 37ms/step - loss: 0.7843 - accuracy: 0.7518


Test loss: 0.7843283414840698
Test accuracy: 0.7517908811569214


In [46]:
print('Test accuracy model with dates:', score[1])
print('Test accuracy model without dates:', score_1[1])
print('Test accuracy model without dates with pretrained embeddings:', score_2[1])
print('Test accuracy model without dates with default embeddings:', score_3[1])

Test accuracy model with dates: 0.7428848147392273
Test accuracy model without dates: 0.7541142106056213
Test accuracy model without dates with pretrained embeddings: 0.7543972730636597
Test accuracy model without dates with default embeddings: 0.7517908811569214


Точность модели с предобученным эмбеддингом на уровне модели с эмбеддингом по умолчанию